In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./datasets/training.csv')
df.head()

,Engagements,Followers at Posting,Created,Type,Description
0,502093,36984682,2019-05-21 23:30:51 EDT,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,2019-05-21 22:53:33 EDT,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,2019-05-21 22:19:58 EDT,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,2019-05-21 22:02:41 EDT,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,2019-05-21 20:47:49 EDT,Video,@giannis_an34 goes baseline early to rock the ...


In [3]:
holdout = pd.read_csv('./datasets/holdout.csv')
holdout.head()

,Engagements,Followers at Posting,Created,Type,Description
0,NaN,36984682,2019-05-22 00:31:55 EDT,Photo,The @raptors even up the Eastern Conference Fi...
1,NaN,36984682,2019-05-21 23:20:41 EDT,Photo,The @raptors even the Eastern Conference Final...
2,NaN,36984682,2019-05-21 21:18:51 EDT,Video,@sergeibaka keeps it alive for the @raptors! (...
3,NaN,36955156,2019-05-21 18:20:09 EDT,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...
4,NaN,36955156,2019-05-21 15:04:31 EDT,Video,?? the @raptors & @bucks in transition! #NBABr...


In [4]:
df.isnull().sum()

Engagements              0
Followers at Posting     0
Created                  0
Type                     0
Description             14
dtype: int64

In [5]:
holdout.isnull().sum()

Engagements             1000
Followers at Posting       0
Created                    0
Type                       0
Description                0
dtype: int64

## Dropping 14 Instagram Posts with No Description 

In [6]:
df.dropna(subset= ['Description'], axis = 0, inplace = True)

In [7]:
df.shape

(7752, 5)

In [8]:
df.isnull().sum().sum()

0

In [9]:
df.dtypes

Engagements              int64
Followers at Posting     int64
Created                 object
Type                    object
Description             object
dtype: object

In [10]:
df['Type'].value_counts()

Video    5451
Photo    1588
Album     713
Name: Type, dtype: int64

## Getting Dummy Column

In [11]:
dataframe = pd.get_dummies(df, columns = ['Type'], drop_first= True )
dataframe_holdout = pd.get_dummies(holdout, columns = ['Type'], drop_first= True )

In [12]:
dataframe.head()

,Engagements,Followers at Posting,Created,Description,Type_Photo,Type_Video
0,502093,36984682,2019-05-21 23:30:51 EDT,The @raptors bench trio of @sergeibaka @norman...,0,1
1,603380,36984682,2019-05-21 22:53:33 EDT,@kyle_lowry7 pulls from deep for the @raptors ...,0,1
2,603380,36984682,2019-05-21 22:19:58 EDT,@k_mid22 with some english on the @bucks dime!,0,1
3,725100,36984682,2019-05-21 22:02:41 EDT,Kawhi punches it home with the left on TNT!,0,1
4,661446,36984682,2019-05-21 20:47:49 EDT,@giannis_an34 goes baseline early to rock the ...,0,1


In [13]:
dataframe_holdout.head()

,Engagements,Followers at Posting,Created,Description,Type_Photo,Type_Video
0,NaN,36984682,2019-05-22 00:31:55 EDT,The @raptors even up the Eastern Conference Fi...,1,0
1,NaN,36984682,2019-05-21 23:20:41 EDT,The @raptors even the Eastern Conference Final...,1,0
2,NaN,36984682,2019-05-21 21:18:51 EDT,@sergeibaka keeps it alive for the @raptors! (...,0,1
3,NaN,36955156,2019-05-21 18:20:09 EDT,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,0,1
4,NaN,36955156,2019-05-21 15:04:31 EDT,?? the @raptors & @bucks in transition! #NBABr...,0,1


In [14]:
import seaborn as sns

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
features = ['Followers at Posting', 'Description', 'Type_Photo', 'Type_Video']
X = dataframe[features]
y = dataframe['Engagements']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
X_train_nlp = X_train[['Description']]
X_holdout_nlp = dataframe_holdout[['Description']]

## Cleaning Up Train Data

In [19]:
X_train_nlp.head()

,Description
6709,A new career-high & @pelicansnba franchise-hig...
1615,Swipe through to recap Wednesday’s seven games!
980,@dloading breaks out an array of moves!
3759,No. 2??...@giannis_an34 ELECTRIFIES The Garden...
6991,@dwighthoward takes it coast to coast for the ...


In [20]:
X_holdout_nlp.head()

,Description
0,The @raptors even up the Eastern Conference Fi...
1,The @raptors even the Eastern Conference Final...
2,@sergeibaka keeps it alive for the @raptors! (...
3,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...
4,?? the @raptors & @bucks in transition! #NBABr...


In [21]:
import string 

## Removing Punctuations and Numbers

In [22]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
remove = list(string.punctuation)

In [24]:
remove.extend(['0','1','2','3','4','5','6','7','8','9'])

In [25]:
from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in remove)

In [26]:
X_train_nlp['no punctuation and numbers'] = X_train_nlp['Description'].apply(strip_punctuation)
X_holdout_nlp['no punctuation and numbers'] = X_holdout_nlp['Description'].apply(strip_punctuation)

/Users/erinhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/erinhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
X_train_nlp.head()

,Description,no punctuation and numbers
6709,A new career-high & @pelicansnba franchise-hig...,A new careerhigh pelicansnba franchisehigh a...
1615,Swipe through to recap Wednesday’s seven games!,Swipe through to recap Wednesday’s seven games
980,@dloading breaks out an array of moves!,dloading breaks out an array of moves
3759,No. 2??...@giannis_an34 ELECTRIFIES The Garden...,No giannisan ELECTRIFIES The Garden BESTofNBA
6991,@dwighthoward takes it coast to coast for the ...,dwighthoward takes it coast to coast for the N...


In [28]:
X_holdout_nlp.head()

,Description,no punctuation and numbers
0,The @raptors even up the Eastern Conference Fi...,The raptors even up the Eastern Conference Fin...
1,The @raptors even the Eastern Conference Final...,The raptors even the Eastern Conference Finals...
2,@sergeibaka keeps it alive for the @raptors! (...,sergeibaka keeps it alive for the raptors nba...
3,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,The SUPERHUMAN HANDS of Giannis Kawhi \r\r b...
4,?? the @raptors & @bucks in transition! #NBABr...,the raptors bucks in transition NBABreakdown...


## Adding Vader

In [29]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [30]:
X_train_nlp['vader'] = [analyser.polarity_scores(text) for text in X_train_nlp['no punctuation and numbers']]
X_holdout_nlp['vader'] = [analyser.polarity_scores(text) for text in X_holdout_nlp['no punctuation and numbers']]

/Users/erinhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/erinhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
X_train_nlp.head()

,Description,no punctuation and numbers,vader
6709,A new career-high & @pelicansnba franchise-hig...,A new careerhigh pelicansnba franchisehigh a...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1615,Swipe through to recap Wednesday’s seven games!,Swipe through to recap Wednesday’s seven games,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
980,@dloading breaks out an array of moves!,dloading breaks out an array of moves,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
3759,No. 2??...@giannis_an34 ELECTRIFIES The Garden...,No giannisan ELECTRIFIES The Garden BESTofNBA,"{'neg': 0.306, 'neu': 0.694, 'pos': 0.0, 'comp..."
6991,@dwighthoward takes it coast to coast for the ...,dwighthoward takes it coast to coast for the N...,"{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compou..."


In [32]:
X_holdout_nlp.head()

,Description,no punctuation and numbers,vader
0,The @raptors even up the Eastern Conference Fi...,The raptors even up the Eastern Conference Fin...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
1,The @raptors even the Eastern Conference Final...,The raptors even the Eastern Conference Finals...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,@sergeibaka keeps it alive for the @raptors! (...,sergeibaka keeps it alive for the raptors nba...,"{'neg': 0.0, 'neu': 0.729, 'pos': 0.271, 'comp..."
3,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,The SUPERHUMAN HANDS of Giannis Kawhi \r\r b...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,?? the @raptors & @bucks in transition! #NBABr...,the raptors bucks in transition NBABreakdown...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


## Pull out all Vader Scores

In [33]:
#this code creates new columns of vader scores 
X_train_nlp['neg'] = X_train_nlp['vader'].apply(lambda x: x.get('neg'))
X_train_nlp['neu'] = X_train_nlp['vader'].apply(lambda x: x.get('neu'))
X_train_nlp['pos'] = X_train_nlp['vader'].apply(lambda x: x.get('pos'))
X_train_nlp['compound'] = X_train_nlp['vader'].apply(lambda x: x.get('compound'))

In [35]:
#this code creates new columns of vader scores 
X_holdout_nlp['neg'] = X_holdout_nlp['vader'].apply(lambda x: x.get('neg'))
X_holdout_nlp['neu'] = X_holdout_nlp['vader'].apply(lambda x: x.get('neu'))
X_holdout_nlp['pos'] = X_holdout_nlp['vader'].apply(lambda x: x.get('pos'))
X_holdout_nlp['compound'] = X_holdout_nlp['vader'].apply(lambda x: x.get('compound'))

/Users/erinhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/erinhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
X_train_nlp.head()

,Description,no punctuation and numbers,vader,neg,neu,pos,compound
6709,A new career-high & @pelicansnba franchise-hig...,A new careerhigh pelicansnba franchisehigh a...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000
1615,Swipe through to recap Wednesday’s seven games!,Swipe through to recap Wednesday’s seven games,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000
980,@dloading breaks out an array of moves!,dloading breaks out an array of moves,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000
3759,No. 2??...@giannis_an34 ELECTRIFIES The Garden...,No giannisan ELECTRIFIES The Garden BESTofNBA,"{'neg': 0.306, 'neu': 0.694, 'pos': 0.0, 'comp...",0.306,0.694,0.0,-0.296
6991,@dwighthoward takes it coast to coast for the ...,dwighthoward takes it coast to coast for the N...,"{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compou...",0.270,0.730,0.0,-0.516


In [37]:
X_holdout_nlp.head()

,Description,no punctuation and numbers,vader,neg,neu,pos,compound
0,The @raptors even up the Eastern Conference Fi...,The raptors even up the Eastern Conference Fin...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000
1,The @raptors even the Eastern Conference Final...,The raptors even the Eastern Conference Finals...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000
2,@sergeibaka keeps it alive for the @raptors! (...,sergeibaka keeps it alive for the raptors nba...,"{'neg': 0.0, 'neu': 0.729, 'pos': 0.271, 'comp...",0.0,0.729,0.271,0.3818
3,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,The SUPERHUMAN HANDS of Giannis Kawhi \r\r b...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000
4,?? the @raptors & @bucks in transition! #NBABr...,the raptors bucks in transition NBABreakdown...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000


## Tokenize then Lemmatize 

In [38]:
import nltk

In [39]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

In [40]:
#instntiate tokenizer
tokenizer = RegexpTokenizer(r'\w+')
lem = WordNetLemmatizer()

In [41]:
def lemmatize(text):
    return [lem.lemmatize(w.lower()) for w in tokenizer.tokenize(text)]  #add .replace("'", "")



In [42]:
X_train_nlp['lemmatized'] = X_train_nlp['no punctuation and numbers'].apply(lemmatize)
X_holdout_nlp['lemmatized'] = X_holdout_nlp['no punctuation and numbers'].apply(lemmatize)

In [43]:
X_train_nlp.head()

,Description,no punctuation and numbers,vader,neg,neu,pos,compound,lemmatized
6709,A new career-high & @pelicansnba franchise-hig...,A new careerhigh pelicansnba franchisehigh a...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000,"[a, new, careerhigh, pelicansnba, franchisehig..."
1615,Swipe through to recap Wednesday’s seven games!,Swipe through to recap Wednesday’s seven games,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000,"[swipe, through, to, recap, wednesday, s, seve..."
980,@dloading breaks out an array of moves!,dloading breaks out an array of moves,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000,"[dloading, break, out, an, array, of, move]"
3759,No. 2??...@giannis_an34 ELECTRIFIES The Garden...,No giannisan ELECTRIFIES The Garden BESTofNBA,"{'neg': 0.306, 'neu': 0.694, 'pos': 0.0, 'comp...",0.306,0.694,0.0,-0.296,"[no, giannisan, electrifies, the, garden, best..."
6991,@dwighthoward takes it coast to coast for the ...,dwighthoward takes it coast to coast for the N...,"{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compou...",0.270,0.730,0.0,-0.516,"[dwighthoward, take, it, coast, to, coast, for..."


In [44]:
X_holdout_nlp.head()

,Description,no punctuation and numbers,vader,neg,neu,pos,compound,lemmatized
0,The @raptors even up the Eastern Conference Fi...,The raptors even up the Eastern Conference Fin...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, raptor, even, up, the, eastern, conferen..."
1,The @raptors even the Eastern Conference Final...,The raptors even the Eastern Conference Finals...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, raptor, even, the, eastern, conference, ..."
2,@sergeibaka keeps it alive for the @raptors! (...,sergeibaka keeps it alive for the raptors nba...,"{'neg': 0.0, 'neu': 0.729, 'pos': 0.271, 'comp...",0.0,0.729,0.271,0.3818,"[sergeibaka, keep, it, alive, for, the, raptor..."
3,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,The SUPERHUMAN HANDS of Giannis Kawhi \r\r b...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, superhuman, hand, of, giannis, kawhi, bu..."
4,?? the @raptors & @bucks in transition! #NBABr...,the raptors bucks in transition NBABreakdown...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, raptor, buck, in, transition, nbabreakdo..."


In [45]:
X_train_nlp['texty'] = [[' '.join(i)] for i in X_train_nlp['lemmatized']]
X_train_nlp['string'] = [str(i) for i in X_train_nlp['texty']]
X_train_nlp['stringOG'] = [i.replace('[', '').replace(']', '').replace("'", '') for i in X_train_nlp['string']]

X_holdout_nlp['texty'] = [[' '.join(i)] for i in X_holdout_nlp['lemmatized']]
X_holdout_nlp['string'] = [str(i) for i in X_holdout_nlp['texty']]
X_holdout_nlp['stringOG'] = [i.replace('[', '').replace(']', '').replace("'", '') for i in X_holdout_nlp['string']]


In [46]:
X_train_nlp.head()

,Description,no punctuation and numbers,vader,neg,neu,pos,compound,lemmatized,texty,string,stringOG
6709,A new career-high & @pelicansnba franchise-hig...,A new careerhigh pelicansnba franchisehigh a...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000,"[a, new, careerhigh, pelicansnba, franchisehig...",[a new careerhigh pelicansnba franchisehigh as...,['a new careerhigh pelicansnba franchisehigh a...,a new careerhigh pelicansnba franchisehigh ass...
1615,Swipe through to recap Wednesday’s seven games!,Swipe through to recap Wednesday’s seven games,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000,"[swipe, through, to, recap, wednesday, s, seve...",[swipe through to recap wednesday s seven game],['swipe through to recap wednesday s seven game'],swipe through to recap wednesday s seven game
980,@dloading breaks out an array of moves!,dloading breaks out an array of moves,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.0,0.000,"[dloading, break, out, an, array, of, move]",[dloading break out an array of move],['dloading break out an array of move'],dloading break out an array of move
3759,No. 2??...@giannis_an34 ELECTRIFIES The Garden...,No giannisan ELECTRIFIES The Garden BESTofNBA,"{'neg': 0.306, 'neu': 0.694, 'pos': 0.0, 'comp...",0.306,0.694,0.0,-0.296,"[no, giannisan, electrifies, the, garden, best...",[no giannisan electrifies the garden bestofnba],['no giannisan electrifies the garden bestofnba'],no giannisan electrifies the garden bestofnba
6991,@dwighthoward takes it coast to coast for the ...,dwighthoward takes it coast to coast for the N...,"{'neg': 0.27, 'neu': 0.73, 'pos': 0.0, 'compou...",0.270,0.730,0.0,-0.516,"[dwighthoward, take, it, coast, to, coast, for...",[dwighthoward take it coast to coast for the n...,['dwighthoward take it coast to coast for the ...,dwighthoward take it coast to coast for the nb...


In [48]:
X_holdout_nlp.head()

,Description,no punctuation and numbers,vader,neg,neu,pos,compound,lemmatized,texty,string,stringOG
0,The @raptors even up the Eastern Conference Fi...,The raptors even up the Eastern Conference Fin...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, raptor, even, up, the, eastern, conferen...",[the raptor even up the eastern conference final],['the raptor even up the eastern conference fi...,the raptor even up the eastern conference final
1,The @raptors even the Eastern Conference Final...,The raptors even the Eastern Conference Finals...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, raptor, even, the, eastern, conference, ...",[the raptor even the eastern conference final ...,['the raptor even the eastern conference final...,the raptor even the eastern conference final w...
2,@sergeibaka keeps it alive for the @raptors! (...,sergeibaka keeps it alive for the raptors nba...,"{'neg': 0.0, 'neu': 0.729, 'pos': 0.271, 'comp...",0.0,0.729,0.271,0.3818,"[sergeibaka, keep, it, alive, for, the, raptor...",[sergeibaka keep it alive for the raptor nbaon...,['sergeibaka keep it alive for the raptor nbao...,sergeibaka keep it alive for the raptor nbaontnt
3,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,The SUPERHUMAN HANDS of Giannis Kawhi \r\r b...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, superhuman, hand, of, giannis, kawhi, bu...",[the superhuman hand of giannis kawhi buck x r...,['the superhuman hand of giannis kawhi buck x ...,the superhuman hand of giannis kawhi buck x ra...
4,?? the @raptors & @bucks in transition! #NBABr...,the raptors bucks in transition NBABreakdown...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0,1.000,0.000,0.0000,"[the, raptor, buck, in, transition, nbabreakdo...",[the raptor buck in transition nbabreakdown ea...,['the raptor buck in transition nbabreakdown e...,the raptor buck in transition nbabreakdown eas...


## Vectorizer Time 

In [49]:
X_train_vec = X_train_nlp['stringOG']
X_holdout_vec = X_holdout_nlp['stringOG']

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
tfidf = TfidfVectorizer(stop_words= 'english')
tfidf.fit(X_train_vec)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [52]:
# transofrming 
train_transform = tfidf.transform(X_train_vec)
holdout_transform = tfidf.transform(X_holdout_vec)

In [53]:
#creating df of transformed data
train_transform_df = pd.DataFrame(train_transform.toarray(),
                   columns= tfidf.get_feature_names())

holdout_transform_df = pd.DataFrame(holdout_transform.toarray(),
                   columns= tfidf.get_feature_names())

In [54]:
train_transform_df.shape

(5193, 3400)

In [55]:
holdout_transform_df.shape

(1000, 3400)

## We finally made our train and test!

In [56]:
vader = ['neg', 'neu', 'pos', 'compound']
X_train_df = pd.concat(objs = [train_transform_df,
                       X_train_nlp.reset_index()[vader],
                       X_train.reset_index()[['Followers at Posting', 'Type_Photo', 'Type_Video']]], 
                       axis = 1)

In [57]:
X_train_df.isnull().sum().sum()

0

In [58]:
vader = ['neg', 'neu', 'pos', 'compound']
X_holdout_df = pd.concat(objs = [holdout_transform_df,
                       X_holdout_nlp.reset_index()[vader],
                       dataframe_holdout.reset_index()[['Followers at Posting', 'Type_Photo', 'Type_Video']]], 
                       axis = 1)

In [59]:
X_holdout_df.isnull().sum().sum()

0

In [60]:
X_train_df.shape

(5193, 3407)

In [61]:
X_holdout_df.shape

(1000, 3407)

In [62]:
X_holdout_df.head()

,aarontaosunitedmasters,abc,abcespn,ability,able,academy,acfresh,achievement,acknowledges,acl,...,zhuri,zmane,zo,neg,neu,pos,compound,Followers at Posting,Type_Photo,Type_Video
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000,0.000,0.0000,36984682,1,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000,0.000,0.0000,36984682,1,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.729,0.271,0.3818,36984682,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000,0.000,0.0000,36955156,0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000,0.000,0.0000,36955156,0,1


In [63]:
X_holdout_df.to_csv('./datasets/X_holdout_df.csv')